# 🐼 Pandas Cookbook & Cheat Sheet

This notebook is a practical **pandas cookbook**: common operations, short explanations, and runnable examples you can tweak for your own data.

In [ ]:
import pandas as pd
import numpy as np
pd.__version__

## Sample DataFrames Used in Examples

In [ ]:
# Simple demo DataFrame
df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'group': ['A', 'A', 'B', 'B', 'B', 'C'],
    'subgroup': ['X', 'Y', 'X', 'Y', 'Y', 'X'],
    'value': [10, 15, 7, 30, 22, 5],
    'qty': [1, 3, 2, 5, 4, 1],
    'category': ['foo', 'bar', 'foo', 'bar', 'foo', 'baz'],
    'date_string': [
        '2025-01-01', '2025-01-03', '2025-01-05',
        '2025-02-01', '2025-02-10', '2025-03-01'
    ]
})

df

---
## 🧾 Creating & Reading Data

Basic ways to create objects and read/write files.

In [ ]:
# From Python objects
s = pd.Series([1, 2, 3], name='col')

df_manual = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': ['a', 'b', 'c']
})

print('Series:')
print(s)
print('\nDataFrame:')
print(df_manual)

In [ ]:
# Reading and writing files (paths commented out)
# df_csv = pd.read_csv('file.csv')
# df_parquet = pd.read_parquet('file.parquet')
# df_excel = pd.read_excel('file.xlsx', sheet_name=0)

# df.to_csv('out.csv', index=False)
# df.to_parquet('out.parquet')
# df.to_excel('out.xlsx', index=False)
print('See comments for read/write examples.')

---
## 👀 Inspecting Data

Quick ways to peek at your DataFrame.

In [ ]:
print('Head:')
display(df.head())

print('Tail:')
display(df.tail(3))

print('Shape:', df.shape)
print('Columns:', df.columns.tolist())
print('\nInfo:')
df.info()

print('\nDescribe:')
display(df.describe())

---
## 🔎 Selecting Columns & Rows

Basic slicing and column selection.

In [ ]:
# Column selection
col = df['value']            # Series
cols = df[['id', 'value']]   # DataFrame

print('Single column (Series):')
print(col)
print('\nTwo columns (DataFrame):')
print(cols)

# Add / overwrite column
df['value_plus_10'] = df['value'] + 10
df = df.assign(ratio=df['value'] / df['qty'])

df.head()

In [ ]:
# Row slices by position (label-agnostic)
df[0:3]

---
## 📍 `.loc` — Label-based Selection

Use index/column **labels**. Slices are inclusive.


In [ ]:
# Ensure we have a custom index for demonstration
df_loc = df.set_index('id')

print('Row with id label 3:')
display(df_loc.loc[3])

print('\nRows with id labels 2..4:')
display(df_loc.loc[2:4])

print('\nRows 2..4, columns group & value:')
display(df_loc.loc[2:4, ['group', 'value']])

# Boolean mask with loc
mask = df_loc['value'] > 10
print('\nRows with value > 10:')
display(df_loc.loc[mask, ['group', 'value']])

---
## 🔢 `.iloc` — Position-based Selection

Use **integer positions** (0-based). End index is exclusive.

In [ ]:
print('First row:')
display(df.iloc[0])

print('\nRows 0..2:')
display(df.iloc[0:3])

print('\nRows 0..2, columns 1..2:')
display(df.iloc[0:3, 1:3])

---
## 🎯 Boolean Indexing & Filtering


In [ ]:
print('Rows where value > 10:')
display(df[df['value'] > 10])

print('value between 10 and 25 OR group in [A, C]:')
display(df[(df['value'].between(10, 25)) | (df['group'].isin(['A', 'C']))])

print('Rows where category is NOT foo:')
display(df[~df['category'].isin(['foo'])])

---
## 📏 Sorting & Ranking


In [ ]:
print('Sort by value descending:')
display(df.sort_values('value', ascending=False))

print('Sort by group asc, value desc:')
display(df.sort_values(['group', 'value'], ascending=[True, False]))

df['value_rank'] = df['value'].rank(method='dense', ascending=False)
print('With value_rank:')
display(df[['id', 'value', 'value_rank']])

---
## 🕳 Handling Missing Data


In [ ]:
# Create a small example with NaNs
df_na = df.copy()
df_na.loc[2, 'value'] = np.nan
df_na.loc[4, 'qty'] = np.nan

print('Missing counts per column:')
print(df_na.isna().sum())

print('\nDrop rows with any NaN:')
display(df_na.dropna())

print('\nFill NaNs in value with mean:')
df_filled = df_na.copy()
df_filled['value'] = df_filled['value'].fillna(df_filled['value'].mean())
display(df_filled)

---
## 👥 `groupby` — Split-Apply-Combine


In [ ]:
print('Mean value by group:')
display(df.groupby('group')['value'].mean())

print('\nSum of value by group & subgroup:')
display(df.groupby(['group', 'subgroup'])['value'].sum())

print('\nMultiple aggregations:')
display(df.groupby('group').agg({
    'value': ['count', 'mean', 'min', 'max'],
    'qty': 'sum'
}))

print('\nUsing as_index=False (flat DataFrame):')
display(df.groupby('group', as_index=False)['value'].sum())

print('\nTransform example: group-wise mean value:')
df['group_mean_value'] = df.groupby('group')['value'].transform('mean')
display(df[['id', 'group', 'value', 'group_mean_value']])

---
## 📊 Pivot Tables & Crosstabs


In [ ]:
print('Pivot table: mean value by group (rows) and subgroup (cols):')
pivot = pd.pivot_table(
    df,
    values='value',
    index='group',
    columns='subgroup',
    aggfunc='mean',
    fill_value=0
)
display(pivot)

print('\nCrosstab: frequency of group vs category:')
ct = pd.crosstab(df['group'], df['category'])
display(ct)

print('\nRow-normalized crosstab (percentages):')
display(pd.crosstab(df['group'], df['category'], normalize='index').round(2))

---
## 🔗 Merging & Joining


In [ ]:
# Example: join df with a small lookup table of group labels
group_labels = pd.DataFrame({
    'group': ['A', 'B', 'C'],
    'group_label': ['Alpha', 'Beta', 'Gamma']
})

merged = df.merge(group_labels, on='group', how='left')
merged.head()

In [ ]:
# Concatenation examples
df_part1 = df.iloc[:3]
df_part2 = df.iloc[3:]

print('Row-wise concat:')
display(pd.concat([df_part1, df_part2], axis=0, ignore_index=True))

print('\nColumn-wise concat (just an example):')
display(pd.concat([df[['id', 'group']], df[['value', 'qty']]], axis=1))

---
## 🔤 String Operations (`.str`)


In [ ]:
names = pd.Series([' Alice ', 'bob', 'CHARLIE', None])
print('Original:')
display(names)

print('\nLower:')
display(names.str.lower())

print('\nStrip:')
display(names.str.strip())

print('\nContains "a" (case-insensitive):')
display(names.str.contains('a', case=False, na=False))

multi = pd.Series(['first last', 'hello world', 'pandas rocks'])
print('\nSplit on space (expand to columns):')
display(multi.str.split(' ', expand=True))

---
## 📅 Datetime Operations


In [ ]:
df_dates = df.copy()
df_dates['date'] = pd.to_datetime(df_dates['date_string'])

df_dates['year'] = df_dates['date'].dt.year
df_dates['month'] = df_dates['date'].dt.month
df_dates['day'] = df_dates['date'].dt.day
df_dates['dow'] = df_dates['date'].dt.day_name()

print('With datetime columns:')
display(df_dates[['id', 'date', 'year', 'month', 'day', 'dow']])

print('\nFilter dates in Feb 2025:')
display(df_dates[(df_dates['date'] >= '2025-02-01') & (df_dates['date'] < '2025-03-01')])

print('\nSet index to date and resample monthly:')
df_resample = df_dates.set_index('date')
display(df_resample['value'].resample('M').mean())

---
## 🏷 Categorical Data


In [ ]:
df_cat = df.copy()
df_cat['category'] = df_cat['category'].astype('category')

print('Categories:')
print(df_cat['category'].cat.categories)

df_cat['category'] = df_cat['category'].cat.add_categories(['new_cat'])
print('\nAfter adding new_cat:')
print(df_cat['category'].cat.categories)

---
## 🧮 Apply, Map, and Vectorization


In [ ]:
df_apply = df.copy()

# Column-wise apply
df_apply['double_value'] = df_apply['value'].apply(lambda x: x * 2)

# Row-wise apply
df_apply['value_plus_qty'] = df_apply.apply(lambda row: row['value'] + row['qty'], axis=1)

# map with dict
mapping = {'A': 'Group A', 'B': 'Group B', 'C': 'Group C'}
df_apply['group_name'] = df_apply['group'].map(mapping)

# Vectorized operation
df_apply['value_per_qty'] = df_apply['value'] / df_apply['qty']

df_apply.head()

---
## 📈 Descriptive Statistics & Quick Analysis


In [ ]:
print('value mean:', df['value'].mean())
print('value std:', df['value'].std())
print('value 95th percentile:', df['value'].quantile(0.95))

print('\nValue counts for group:')
print(df['group'].value_counts())

print('\nCorrelation (numeric columns):')
display(df.corr(numeric_only=True))

---
## 🔄 Reshaping: `melt`, `stack`, `unstack`


In [ ]:
# Create a small wide-form DataFrame
sales = pd.DataFrame({
    'id': [1, 2, 3],
    'Jan': [100, 150, 80],
    'Feb': [90, 160, 120]
})
print('Original wide sales:')
display(sales)

print('\nMelt to long format:')
long_sales = pd.melt(
    sales,
    id_vars=['id'],
    value_vars=['Jan', 'Feb'],
    var_name='month',
    value_name='amount'
)
display(long_sales)

# stack / unstack example
stack_example = long_sales.set_index(['id', 'month'])['amount']
print('\nStacked (Series with MultiIndex):')
display(stack_example)

print('\nUnstack month back to columns:')
display(stack_example.unstack('month'))

---
## 🪟 Window Functions (Rolling / Expanding)


In [ ]:
df_win = df_dates.sort_values('date').set_index('date')

df_win['roll_mean_3'] = df_win['value'].rolling(window=3, min_periods=1).mean()
df_win['roll_sum_3'] = df_win['value'].rolling(window=3, min_periods=1).sum()

df_win['cum_sum'] = df_win['value'].expanding().sum()
df_win['cum_mean'] = df_win['value'].expanding().mean()

df_win[['value', 'roll_mean_3', 'roll_sum_3', 'cum_sum', 'cum_mean']]

---
## 🧹 Duplicates & Basic Cleaning


In [ ]:
df_dup = pd.concat([df.iloc[:3], df.iloc[:3]], ignore_index=True)

print('With duplicates:')
display(df_dup)

print('\nDuplicated rows (boolean):')
print(df_dup.duplicated())

print('\nAfter drop_duplicates:')
display(df_dup.drop_duplicates())

# Rename columns
renamed = df.rename(columns={'value': 'value_renamed'})
print('\nRenamed columns:')
display(renamed.head())

# Reset index
reset = df.set_index('id').reset_index()
print('\nReset index example:')
display(reset.head())

---
## 🪄 Common Handy Idioms


In [ ]:
df_idiom = df.copy()

# Conditional assignment
df_idiom['size'] = 'big'
df_idiom.loc[df_idiom['value'] < 10, 'size'] = 'small'

# Top N
top3 = df_idiom.nlargest(3, 'value')
bottom3 = df_idiom.nsmallest(3, 'value')

print('Top 3 by value:')
display(top3)

print('Bottom 3 by value:')
display(bottom3)

# Method chaining example
summary = (
    df_idiom
    .query('value > 0')
    .assign(log_value=lambda d: np.log(d['value']))
    .groupby('group', as_index=False)['log_value']
    .mean()
    .sort_values('log_value', ascending=False)
)

print('Summary with chaining:')
display(summary)